In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adityamiskin","key":"f7e085bcb86c988a48c21b3d3b2db389"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c digit-recognizer

 82% 5.00M/6.09M [00:01<00:00, 4.25MB/s]
100% 6.09M/6.09M [00:01<00:00, 5.04MB/s]
 55% 5.00M/9.16M [00:00<00:00, 18.3MB/s]
100% 9.16M/9.16M [00:00<00:00, 30.6MB/s]
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 214MB/s]


In [6]:
from zipfile import ZipFile
file_name='/content/test.csv.zip'
with ZipFile(file_name,'r') as zp:
  zp.extractall()
print("done")

done


In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from tensorflow import keras
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [30]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
Y_train = train_df['label']
X_train = train_df.drop('label',axis=1).copy()
X_test = test_df

In [53]:
X_train = X_train/255.0
X_test = X_test/255.0

In [54]:
X_train = X_train.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)

In [55]:
Y_train = to_categorical(Y_train, num_classes = 10)

In [56]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=2)

In [57]:
datagen = ImageDataGenerator(rotation_range=10,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             horizontal_flip=False,
                             vertical_flip=False)
datagen.fit(X_train)

In [58]:
model = keras.Sequential([keras.layers.Conv2D(64,(3,3),activation='relu'),
                                 keras.layers.MaxPooling2D(2,2),
                                 keras.layers.Conv2D(64,(3,3),activation='relu'),
                                 keras.layers.MaxPooling2D(2,2),
                                 keras.layers.Dropout(0.1),
                                 keras.layers.Flatten(),
                                 keras.layers.Dense(256,activation='relu'),
                                 keras.layers.Dropout(0.5),
                                 keras.layers.Dense(10,activation='softmax')])

In [59]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [60]:
history = model.fit_generator(datagen.flow(X_train,Y_train,batch_size=86),
                              validation_data=(X_val,Y_val),
                              verbose=1,
                              epochs=30,
                              steps_per_epoch=86)

Epoch 1/30
86/86 [==============================] - 3s 30ms/step - loss: 1.2076 - accuracy: 0.5972 - val_loss: 0.2328 - val_accuracy: 0.9326
Epoch 2/30
86/86 [==============================] - 2s 29ms/step - loss: 0.4380 - accuracy: 0.8659 - val_loss: 0.1346 - val_accuracy: 0.9611
Epoch 3/30
86/86 [==============================] - 2s 28ms/step - loss: 0.3313 - accuracy: 0.8976 - val_loss: 0.1202 - val_accuracy: 0.9632
Epoch 4/30
86/86 [==============================] - 2s 28ms/step - loss: 0.2661 - accuracy: 0.9167 - val_loss: 0.0945 - val_accuracy: 0.9704
Epoch 5/30
86/86 [==============================] - 2s 28ms/step - loss: 0.2224 - accuracy: 0.9316 - val_loss: 0.0878 - val_accuracy: 0.9732
Epoch 6/30
86/86 [==============================] - 2s 29ms/step - loss: 0.2008 - accuracy: 0.9371 - val_loss: 0.0813 - val_accuracy: 0.9758
Epoch 7/30
86/86 [==============================] - 2s 28ms/step - loss: 0.1910 - accuracy: 0.9410 - val_loss: 0.0672 - val_accuracy: 0.9790
Epoch 8/30
86

In [64]:
result = model.predict(X_test)
result = np.argmax(result,axis=1)
result = pd.Series(result,name="Label")

In [66]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),result],axis = 1)

submission.to_csv("submission.csv",index=False)